# PyQGIS: Expanding QGIS's functionality with Python. 
# Day 2 – Processing and plugins

Yesterday, you learned the basics of PyQGIS: how to run code from the embedded Python console, how to use the central classes (such as QgsVectorLayer) and run operations of layer geometries and attributes. However, the strength of QGIS is its vast library of geospatial algorithms (both native and those added by plugins, such as GRASS) coupled with general data crunching tools. 

In these practicals, you will learn how to run processing algorithms via PyQGIS, even chaining multiple operations together and running batch processes on multiple layers. You will create automated processes using the graphic Model Builder and expand on the models with Python. The point of these exercises is to learn how to use Python in QGIS to create reproducible and easily shareable tools for automated spatial analysis.

The final section is a brief look at QGIS plugin development – what are plugins, when to develop them, what are the basic parts of plugins and the underlying tools, like the QT framework. There's also a challenge related to modifying and adding to a simple graphical plugin.

### Preparations
Open QGIS and load in all the layers from *practical_data.gpkg*. Do this via the GUI or, if you want to do the same programmatically, use the script below. Adapted [from the PyQGIS Cookbook](https://docs.qgis.org/latest/en/docs/pyqgis_developer_cookbook/cheat_sheet.html#layers).

In [ ]:
# EXAMPLE PATH: define the actual path on your system
gpkg_path = "C:/Users/tatu/pyqgis_practical/data/practical_data.gpkg" # windows
gpkg_layer = QgsVectorLayer(gpkg_path, "whole_gpkg", "ogr")

# returns a list of strings describing the sublayers
sub_strings = gpkg_layer.dataProvider().subLayers()
# EXAMPLE: 1!!::!!Paavo!!::!!3027!!::!!MultiPolygon!!::!!geom!!::!!
# !!::!! separates the values

for sub_string in sub_strings:
    layer_name = sub_string.split(gpkg_layer.dataProvider().sublayerSeparator())[1]
    uri = "{0}|layername={1}".format(gpkg_path, layer_name)
    # Create layer
    sub_vlayer = QgsVectorLayer(uri, layer_name, 'ogr')
    # Add layer to map
    if sub_vlayer.isValid():
        QgsProject.instance().addMapLayer(sub_vlayer)
    else:
        print("Can't add layer", layer_name)

Next, make sure you have the processing toolbox open in the GUI. If not, open it from the drop down menu _Processing > Toolbox_. If the toolbox is unavailable for some reason, [follow these instructions](https://docs.qgis.org/3.22/en/docs/training_manual/processing/set_up.html) to remedy.

### Data description
As are reminder, below are brief descriptions of the layers and relevant fields used in these practicals. The original datasets are available through Statistics Finland: here, the field names have been translated to English.

| Layer    | Layer description |Fields | Field description | Source |
| ----------- | ----------- | ----------- | ----------- |  ----------- |
| **Helsinki_region_pop_squares**  |  Capital region population (2020) in 1 km x 1 km squares  |        || [Statistics Finland](https://www.paikkatietohakemisto.fi/geonetwork/srv/eng/catalog.search#/metadata/a901d40a-8a6b-4678-814c-79d2e2ab130c), CC-BY 4.0|
|      |       | pop        |Population in 2020||
|      |       | men / women        | Number of men or women| | 
|      |       | ages x_y        | Number of people in age group|| 
|  **NUTS2_FIN_POP**    |   Population (2020) in NUTS2 regions (suuralueet)    |       ||[Statistics Finland](https://www.paikkatietohakemisto.fi/geonetwork/srv/eng/catalog.search#/metadata/d136a588-7fc6-4865-8d7c-b50d1398e728), CC-BY 4.0|
|      |       | nimi / name        | Name of the region||
|      |       | pop        | Population in 2020||
|  |  | total_pop_perc        | This region's share of total Finnish population||
|  |  | male/female_perc        | Share of men or women in this region||
| **Paavo** | Socio-economic statistics on postal regions |         || [Statistics Finland](https://www.stat.fi/tup/paavo/index_en.html), CC-BY 4.0 |
|  |  | Too many to describe        | [See here for full description](https://www.stat.fi/static/media/uploads/tup/paavo/paavo_kuvaus_en.pdf) ||

### Table of contents:
[**1. Processing algorithms**](#1.-Processing-algorithms)

[1.1. Basics of processing algorithms](#1.1.-Basics-of-processing-algorithms)

[Task 1](#TASK-1)

[1.2. Running processing algorithms with PyQGIS](#1.2.-Running-processing-algorithms-with-PyQGIS)

[1.2.1. Batch processing](#1.2.1.-Batch-processing)

[1.2.2. Chaining algorithms](#1.2.2.-Chaining-algorithms)

[Task 2](#TASK-2)

[1.3. Recreating simplify_and_trim as a processing script](#1.3.-Recreating-simplify_and_trim-as-a-processing-script)

[1.3.1 Graphical processing models](#1.3.1-Graphical-processing-models)

[1.3.2. Building the script base as a graphical model](#1.3.2.-Building-the-script-base-as-a-graphical-model)

[1.3.3. Understanding processing scripts](#1.3.3.-Understanding-processing-scripts)

[1.3.4. Creating a new processing script](#1.3.4.-Creating-a-new-processing-script)

[TASK 3: Finalizing the model](#TASK-3:-Finalizing-the-model)

[1.4. Wrapping up processing](#1.4.-Wrapping-up-processing)

[**Challenge X: Processing algorithm with the @alg decorator**](#Challenge-X:-Processing-algorithm-with-the-@alg-decorator)


[**2. A look at plugin development**](#2.-A-look-at-plugin-development)

[2.1. Plugins: what are they good for?](#2.1.-Plugins:-what-are-they-good-for?)

[2.2. QT framework: signals and slots](#2.2.-QT-framework:-signals-and-slots)

[2.3. Elements of a QGIS plugin](#2.3.-Elements-of-a-QGIS-plugin)

[**Challenge Y: Modifying a plugin**](#Challenge-Y:-Modifying-a-plugin)

## 1. Processing algorithms
## 1.1. Basics of processing algorithms
The processing framework is a collection of helpful functions for topics ranging from network analysis to raster terrain analysis. Both native and custom algorithms can be called programmatically, but doing so requires us to know a few things, namely the algorigthm name and its parameters.

The simplest way to access these is by letting QGIS create the initial code for us.

### TASK 1
- Using the GUI, run _Simplify_ algorithm, under _Vector geometry_. The easiest way to find it is to use the search bar. 
    - Change _Tolerance_ to 5000, otherwise keep the default settings.
    - Run the algorithm. It should add a new memory layer with simplified geometry to the project.
    
![Simplify algorithm in gui](images/simplify_algorithm_settings.JPG)

After running the algorithm, click on the _Processing_ drop-down menu and select _History_. Alternatively, _History_ can be opened through this button on the processing toolbox 

![History button](images/history_button.JPG)

The opened window contains a list of previous processing runs (under ALGORITHM). 

Select the latest one. An algorithm call is displayed on the text box below the list and it looks something like this (formatted for clarity):

In [ ]:
processing.run("native:simplifygeometries", 
               {'INPUT':'C:/Users/tatu/pyqgis_practical/data/practical_data.gpkg|layername=NUTS2_FIN_pop',
                'METHOD':0,
                'TOLERANCE':5000,
                'OUTPUT':'TEMPORARY_OUTPUT'})

What we have here is a call to the processing framework. Specifically, we name the algorithm and pass a dictionary of algorithm parameters (if you need a refresher of Python dicts, [check this out](https://www.w3schools.com/python/python_dictionaries.asp)). 

Notice that the parameters are identical to the ones we defined graphically. Though you might wonder how one could know that _Distance: Douglas-Peucker_ maps to method 0 without first running the algorithm graphically. The processing framework has a function for describing algorithms:

In [ ]:
>>> processing.algorithmHelp("native:simplifygeometries")

In [ ]:
Simplify (native:simplifygeometries)

This algorithm simplifies the geometries in a line or polygon layer. 
---
METHOD: Simplification method

	Parameter type:	QgsProcessingParameterEnum

	Available values:
		- 0: Distance (Douglas-Peucker)
		- 1: Snap to grid
		- 2: Area (Visvalingam)
---

There we have it. Use this function if you want to learn more about the algorithms and the parameters.

However, to use the helper function, you need the algorithm's id string, which is different from its screen name (like _native:simplifygeometries_ vs. _Simplify_). Print all available algorithms [like this](https://docs.qgis.org/latest/en/docs/user_manual/processing/console.html#calling-algorithms-from-the-python-console). Because the hundreds of algorithms can be overwhelming, a simple filter is applied below:

In [ ]:
>>> for alg in QgsApplication.processingRegistry().algorithms():
...        search_str = "simplify"
...        if search_str in alg.displayName().lower():
...            print(alg.displayName(), "->", alg.id())

Simplify Network -> NetworkGT:Simplify Network
Simplify -> native:simplifygeometries

The part before the colon (_NetworkGT_ and _native_) refers to the algorithm provider – in this case a plugin and the native processing library.

## 1.2. Running processing algorithms with PyQGIS
If you ran the processing.run call before, you might've noticed that it simply returned a dictionary with a layer as the value – this is because an algorithm can have multiple outputs. The output objects are accessed by entering the key of the output: most often, that's 'OUTPUT'.

In [ ]:
>>> input_layer_path = 'C:/Users/tatu/pyqgis_practical/data/practical_data.gpkg|layername=NUTS2_FIN_pop'
>>> results = processing.run("native:simplifygeometries", 
...              {'INPUT':input_layer_path,
...                'METHOD':0,
...                'TOLERANCE':5000,
...                'OUTPUT':'TEMPORARY_OUTPUT'})
>>> print(results)
{'OUTPUT': <QgsVectorLayer: 'Simplified' (memory)>}
>>> simplified_layer = results['OUTPUT']

The same can be achieved with one line of code by appending the output key to the end of the processing call. Like this:

In [ ]:
>>> simplified_layer = processing.run("native:simplifygeometries", 
...              {'INPUT':input_layer_path,
...                'METHOD':0,
...                'TOLERANCE':5000,
...                'OUTPUT':'TEMPORARY_OUTPUT'})['OUTPUT'] #<--- output fetched directly

Or alternatively, to load the layer directly to the current project, use `runAndLoadResults`. 


In [ ]:
>>> processing.runAndLoadResults("native:simplifygeometries", 
                {'INPUT': input_layer_path,
                'METHOD':0,
                'TOLERANCE':5000,
                'OUTPUT':'TEMPORARY_OUTPUT'})

### 1.2.1. Batch processing
Running processing algorithms on multiple layers is straightforward to do with Python loops.

The script below runs simplification on all vector layers in a project:

In [ ]:
proj_layers = QgsProject.instance().mapLayers()

for layer in proj_layers.values():
    # excluding other layer types
    if isinstance(layer, QgsVectorLayer):
        processing.runAndLoadResults("native:simplifygeometries", 
                {'INPUT': layer,
                'METHOD':0,
                'TOLERANCE':5000,
                'OUTPUT':'TEMPORARY_OUTPUT'})

### 1.2.2. Chaining algorithms
The real power of these algorithms gets unleashed when they're chained together to form an analysis pipeline. For example, you may remember how long and cumbersome the script used to get to simplify the geometries and fields of our input layer was previously. With the processing framework, we can offset the heavy lifting to two processing algorithms (_Simplify_ and _Drop fields_). Since we're _dropping_ fields instead of _keeping_ them, we need to do a bit of Python magic first.

P.S. If we'd be using ≥QGIS 3.18., we could use [_Retain fields_](https://qgis.org/en/site/forusers/visualchangelog318/#feature-add-retain-fields-algorithm). It's not available in the current LTR.

In [ ]:
# defining input parameters

# path to the NUTS2 layer
input_layer_path = gpkg_path + '|layername=NUTS2_FIN_pop'

input_layer = QgsVectorLayer(input_layer_path, "input_layer", "ogr")

tolerance = 5000
# list of field names to keep
fields_to_keep = ['name', 'pop']

# get all fields
all_fields = input_layer.fields().names()

# BASICALLY: create a list containing all fields except those that are in the "keep" list
drop_fields = [field for field in all_fields if field not in fields_to_keep]

# NOTE! Processing result is a dictionary, 
# here the result layer is fetched by key OUTPUT from the dictionary. 
simplified_layer = processing.run("native:simplifygeometries", 
              {'INPUT':input_layer,
               'METHOD':0,
               'TOLERANCE':tolerance,
               'OUTPUT':'TEMPORARY_OUTPUT'})['OUTPUT'] 

processing.runAndLoadResults("qgis:deletecolumn", 
               {'INPUT':simplified_layer,
                'COLUMN':drop_fields,
                'OUTPUT':'TEMPORARY_OUTPUT'})

Notice that with _runAndLoadResults_, the layer will be named automatically according to the algorithm definitions (like _Remaining fields_).

### TASK 2
- Modify the script above by adding one more algorithm to the pipeline, namely _Add geometry attributes_
    - Find out the algorithm id of _Add geometry attributes_.
    - Create the parameter dictionary as needed for the algorithm (HINT: if you're lost, run the algorithm through the GUI first)
    - Add the algorithm call to the end of the script and modify the previous algorithm call accordingly.

## 1.3. Recreating simplify_and_trim as a processing script

Processing scripts like above are already quite neat, but they do have some weaknesses. Using them requires some programming understanding, which hinders their usability if you'd want to share your tools with others. It's much more user friendly to select the input parameters graphically like in the processing toolbox algorithms.

We will create such a processing tool. One approach would be to write it in code from scratch, like the existing tools are. However, in the interested of time, we will create the base of the script using the graphic _Model Designer_ in QGIS. 

### 1.3.1 Graphical processing models
Model designer enables defining inputs and chaining processing algorithms graphically. The picture below shows an example pipeline for creating a population heatmap clipped to sea boundaries. Subpicture (1) shows the pipeline running through centroid creation, KDE, and clipping with a mask layer. The yellow rectangles are inputs given by the user. These are shown as options when running the script, as seen in subpicture (2). The process outputs a raster (subpicture (3)).

If you want to play around with this model, find it in *scripts > heatmap_from_pop_grid.model3*  in the practical materials.

![Graphic modeler example](images/graphic_model.png)

Great thing about the models is that they can be exported to Python code. We will use this function later on. 

### 1.3.2. Building the script base as a graphical model
Now, let's once again recreate the layer simplification and trimming script, this time as an installable processing tool.

**Open the Model Designer window** from the the left-most button under _Processing Toolbox > Create new Model_.

![Opening graphical model](images/opening_graphical_modeler.JPG)

A mostly empty window is opened. The empty area in the middle is where we'll start building our script. On the left, there's a selection of inputs (1) and algorithms (2), which can be dragged to the builder. Other important functions are naming the model (3), exporting to a Python script (4) and running the model (5).

![Model builder introduction](images/model_builder_intro.png)

Start by dragging and dropping inputs. Remember which ones we defined previously, namely vector layer and a list of fields:

In [ ]:
input_layer_path = gpkg_path + '|layername=NUTS2_FIN_pop'
fields_to_keep = ['name', 'pop']

Equivalently, first drag _Vector layer_, then _Vector field_. Below are the parameters definitions for both. Make sure to toggle _Accept multiple fields_ for the field input:
![Vector layer and field model inputs](images/model_builder_vector_input.png)

Next, click the algorithms tab active. The whole algorithm toolbox and a search function is available. Search for _Drop fields_ and drag it to the model.

In the properties window that opens, change the input type to _Model input_ for both _Input layer_ and _Fields to drop_. Also write _Kept fields_ in the output box:
![Setting drop field algorithm properties](images/drop_field_parameters.png)

Now you should have something like this:

![Opening script template](images/drop_fields_model.JPG)

This is a fully functional model that gives the user an option to select a layer and fields in it, and the drops the selected fields. Hmm, but we want to keep the fields. To implemented that, we need to apply the custom Python logic created before and create a new tool.

Press the button with Python logo (_Export as script algorithm_). It throws a Script editor window where the model is expressed as Python code. Let's first briefly explore the code to understand processing scripts.

### 1.3.3. Understanding processing scripts

The first thing you might notice are the imports. The Python console imports qgis.core automatically, but the same is not true of processing scripts. Following good coding practices, only the necessary methods are imported.

In [ ]:
from qgis.core import QgsProcessing
from qgis.core import QgsProcessingAlgorithm
from qgis.core import QgsProcessingMultiStepFeedback
from qgis.core import QgsProcessingParameterVectorLayer
from qgis.core import QgsProcessingParameterField
from qgis.core import QgsProcessingParameterFeatureSink
import processing

Next, the script defines a **class** that inherits from the general [QgsProcessingAlgorithm](https://qgis.org/pyqgis/3.22/core/QgsProcessingParameterField.html#qgis.core.QgsProcessingParameterField).

In [ ]:
class Model(QgsProcessingAlgorithm):

Under this class, there are methods that both define metadata information, such as the algorithm identifier and display name, and run the actual processing. For the interested, [here's a thorough description of all the mandatory methods](https://docs.qgis.org/latest/en/docs/user_manual/processing/scripts.html#extending-qgsprocessingalgorithm).

The script is currently generically named "model". **Change the name to _Keep fields_** under _name_, _displayName_, _createInstance_ and class definition as shown below
![Script name changes](images/keep_fields_script_names.png)

The workhorse methods are _initAlgorithm_ and _processAlgorithm_. In the initiation step, the input **and output** [parameters](https://qgis.org/pyqgis/latest/core/QgsProcessingParameters.html) are defined.  Notice that the settings are the same we did previously graphically.

BTW, the order the parameters are written here defines the order they're shown in the program. Therefore, if you want the user to first select the layer and then the fields, insert QgsProcessingParameterVectorLayer first.

In [ ]:
def initAlgorithm(self, config=None):
    self.addParameter(QgsProcessingParameterVectorLayer('Inputlayer', 'Input layer', defaultValue=None))
    
    self.addParameter(QgsProcessingParameterField('Fieldstokeep', 'Fields to keep', 
                                                  type=QgsProcessingParameterField.Any, 
                                                  parentLayerParameterName='Inputlayer', 
                                                  allowMultiple=True, defaultValue=None))
    
    self.addParameter(QgsProcessingParameterFeatureSink('KeptFields', 'Kept fields', 
                                                        type=QgsProcessing.TypeVectorAnyGeometry, 
                                                        createByDefault=True, supportsAppend=True, 
                                                        defaultValue=None))

Currently, the processing simply defines the parameters for Drop fields algorithm, runs it and returns a result dictionary.

In [ ]:
def processAlgorithm(self, parameters, context, model_feedback):
    # Use a multi-step feedback, so that individual child algorithm progress reports are adjusted for the
    # overall progress through the model
    feedback = QgsProcessingMultiStepFeedback(1, model_feedback)
    results = {}
    outputs = {}

    # Drop field(s)
    alg_params = {
        'COLUMN': parameters['Fieldstokeep'],
        'INPUT': parameters['Inputlayer'],
        'OUTPUT': parameters['KeptFields']
    }
    
    outputs['DropFields'] = processing.run('qgis:deletecolumn', alg_params, 
                                           context=context, feedback=feedback, 
                                           is_child_algorithm=True)
    results['KeptFields'] = outputs['DropFields']['OUTPUT']
    return results

### 1.3.4. Creating a new processing script
Let's start inserting our own code. First, we need the vector layer and a list of fields to keep. The pre-made code calls the parameter dictionary, like:

In [ ]:
'INPUT': parameters['Inputlayer']

But this returns a _string_ by default, not the objects we need. QgsProcessingAlgorithm has [methods to return the actual objects](https://qgis.org/pyqgis/latest/core/QgsProcessingAlgorithm.html#qgis.core.QgsProcessingAlgorithm.parameterAsVectorLayer)

In [ ]:
input_layer = self.parameterAsVectorLayer(parameters, "Inputlayer", context)
fields_to_keep = self.parameterAsFields(parameters, 'Fieldstokeep', context)

Next, insert the field selection code used previously:

In [ ]:
all_fields = input_layer.fields().names()
        
drop_fields = [field for field in all_fields if field not in fields_to_keep]

Now that drop_fields contains the fields we want to delete, change the COLUMN paramater to drop_fields:

In [ ]:
alg_params = {
    'COLUMN': drop_fields,
    'INPUT': parameters['Inputlayer'],
    'OUTPUT': parameters['KeptFields']
}

All in all, the script looks like this:

In [ ]:
from qgis.core import QgsProcessing
from qgis.core import QgsProcessingAlgorithm
from qgis.core import QgsProcessingMultiStepFeedback
from qgis.core import QgsProcessingParameterVectorLayer
from qgis.core import QgsProcessingParameterField
from qgis.core import QgsProcessingParameterFeatureSink
import processing


class KeepFields(QgsProcessingAlgorithm):

    def initAlgorithm(self, config=None):
        self.addParameter(QgsProcessingParameterVectorLayer('Inputlayer', 'Input layer', defaultValue=None))
        self.addParameter(QgsProcessingParameterField('Fieldstokeep', 'Fields to keep', type=QgsProcessingParameterField.Any, parentLayerParameterName='Inputlayer', allowMultiple=True, defaultValue=None))
        self.addParameter(QgsProcessingParameterFeatureSink('KeptFields', 'Kept fields', type=QgsProcessing.TypeVectorAnyGeometry, createByDefault=True, supportsAppend=True, defaultValue=None))

    def processAlgorithm(self, parameters, context, model_feedback):
        # Use a multi-step feedback, so that individual child algorithm progress reports are adjusted for the
        # overall progress through the model
        feedback = QgsProcessingMultiStepFeedback(1, model_feedback)
        results = {}
        outputs = {}

        input_layer = self.parameterAsVectorLayer(parameters, "Inputlayer", context)
        fields_to_keep = self.parameterAsFields(parameters, 'Fieldstokeep', context)
        
        all_fields = input_layer.fields().names()
        
        drop_fields = [field for field in all_fields if field not in fields_to_keep]

        # Drop field(s)
        alg_params = {
            'COLUMN': drop_fields,
            'INPUT': parameters['Inputlayer'],
            'OUTPUT': parameters['KeptFields']
        }
        outputs['DropFields'] = processing.run('qgis:deletecolumn', alg_params, context=context, feedback=feedback, is_child_algorithm=True)
        results['KeptFields'] = outputs['DropFields']['OUTPUT']
        return results

    def name(self):
        return 'keepFields'

    def displayName(self):
        return 'Keep fields'

    def group(self):
        return ''

    def groupId(self):
        return ''

    def createInstance(self):
        return KeepFields()

**Save your script** locally for example as keep_fields.py.

After saving, close the script window. From prosessing toolboxes' Python drop-down, **select _Add script to toolbox_ and add the script file**. This adds _Keep fields_ in the toolbox under _Scripts_. You may run the tool to test that it indeed works.

![Adding script to toolbox](images/add_script_to_toolbox.JPG)

### TASK 3: Finalizing the model
- Re-create the simplify-and-trim algorithm with the graphical modeler using _Simplify_ and _Keep fields_. It should look something like the pic below.
    - NOTE! Use _Algorithm output_ as the input layer for the second algorithm.
    - Simplification tolerance is simply a _Number_ input.
    - To use the model click `Run model` button (green arrow) in upper toolbar and fill in inputs as you wish. 
    - If the input fields are not in logical order (fields before layer), adjust this under menu `Model` -> `Reorder Model Inputs`. 

![Final model](images/simplify_and_trim_final.png)

You may save and add the finalized model to the toolbox to use it anytime and in other processing scripts!



## 1.4. Wrapping up processing
The take-home messages of this session were:
- Processing algorithms can be run, chained and expanded upon with PyQGIS to create efficient automated GIS processes.
- New processing algorithms can be created by using the model builder – these models can be expanded with Python.

While this tutorial used graphic modeler to create a basis for the scripts, new processing scripts can be created purely by code as well. 

To do this, see the upper row tools of _Processing toolbox_, click on the Python drop-down menu and select _Create new script from template_. [See the user manual for a tutorial](https://docs.qgis.org/latest/en/docs/user_manual/processing/console.html#creating-scripts-and-running-them-from-the-toolbox). Another example is given in Challenge X.

![Opening script template](images/opening_script_template.JPG)

## Challenge X: Processing algorithm with the @alg decorator
There's a way to create processing scripts that takes away a lot of the empty "boilerplate" code. It uses a Python trick called [decorators](https://www.programiz.com/python-programming/decorator): the inputs and outputs are defined by calls to @alg. See [here for a full explanation and an example](https://docs.qgis.org/3.22/en/docs/user_manual/processing/scripts.html#the-alg-decorator).

The code below defines an algorithm that:
 - Gets a polygon input and a value field, such as population, from the same layer
 - Creates as many points inside the layer as the value field's attribute is for that feature
 - Scales the amount of point by a scaling factor given as input (e.g. population for a region is 1500 and scaling factor is 10 --> 1500 / 10 = 150 points). It does this by creating a [_QgsProperty_](https://qgis.org/pyqgis/3.16/core/QgsProperty.html) from an expression.
 - Outputs the point layer _and_ a count of how many features there are in the input layer
 
Such an algorithm could for example be used to visualize the population in an area..

### CHALLENGE X TASKS
1. There's an unused output called SUM_OF_FIELD. Use aggregation (check the first day for a refresher) to get the sum of values in the given value field: append the value to the results dictionary.

2. Add another algorithm of your choice after the points generation and use the points as inputs. The algorithm could for example be _Buffer_ or _Rasterize_. Return both the outputs both processes. [See here for an example](https://docs.qgis.org/3.22/en/docs/user_manual/processing/scripts.html#the-alg-decorator).

In [ ]:
from qgis import processing
from qgis.processing import alg
from qgis.core import QgsProject, QgsProperty

# INPUTS ARE DEFINED HERE
@alg(name='polygonToAggregatedPoints', label='Polygons to aggregated points',
     group='examplescripts', group_label='Example scripts')
# 'INPUT' is the recommended name for the main input parameter
@alg.input(type=alg.SOURCE, name='INPUT', label='Input vector layer')
# 'OUTPUT' is the recommended name for the main output parameter
@alg.input(type=alg.VECTOR_LAYER_DEST, name='OUTPUT',
           label='Point output')
@alg.input(type=alg.FIELD, name='VALUE_FIELD', parentLayerParameterName="INPUT",
           label='Value field to scale on')
@alg.input(type=alg.NUMBER, name='SCALE_VALUE', label='Scale value',
           default=10)
@alg.output(type=alg.NUMBER, name='NUMBER_OF_FEATURES',
            label='Number of features processed')
#@alg.output(type=alg.NUMBER, name='SUM_OF_FIELD',
#            label='Sum of value field')   

def bufferrasteralg(self, parameters, context, feedback, inputs):
    """
    Description of the algorithm.
    (If there is no comment here, you will get an error)
    """
    input_layer = self.parameterAsVectorLayer(parameters,
                                                     'INPUT', context)
    value_field = self.parameterAsString(parameters, 'VALUE_FIELD', context)
    numfeatures = input_layer.featureCount()

    scale_value = self.parameterAsDouble(parameters, 'SCALE_VALUE',
                                            context)
                                            
    points_expression = QgsProperty.fromExpression('round(  "{0}" / {1} )'.format(value_field, str(scale_value)))
    if feedback.isCanceled():
        return {}
    

    points_generation = processing.run("native:randompointsinpolygons", 
                        {'INPUT':parameters['INPUT'],
                        'POINTS_NUMBER': points_expression,
                        'MIN_DISTANCE':0,
                        'OUTPUT': parameters['OUTPUT']},
                        context=context,
                        feedback=feedback,
                        is_child_algorithm=True)

 
    if feedback.isCanceled():
        return {}
    
    results = {'OUTPUT': points_generation['OUTPUT'],
                'NUMBER_OF_FEATURES': numfeatures}
    return results

# 2. A look at plugin development
This section serves as a brief introduction to plugins in QGIS. First, there's a discussion on why plugins are made through a few examples. Then the basic building blocks of a plugin are introduced. Throughout, there're links to various resources where you may read more on specific aspects of plugin development, if you're interested in engaging it yourself. The section is followed by a challenge where you may modify and examine a simple pre-made plugin.

## 2.1. Plugins: what are they good for?
QGIS consists both of [core plugins](https://docs.qgis.org/3.22/en/docs/user_manual/plugins/plugins.html#core-and-external-plugins), maintained by the program developers themselves, and external plugins by independent developers. So, plugins extend the base program in some way. But what does this mean in practice?

A lot of times the plugins integrate some external service's functionality to QGIS: for example by allowing flexible download of rasters from [GeoCubes Finland](https://github.com/geoportti/GeoCubes-Finland-QGIS-Plugin) or to [add resources](https://qgis-contribution.github.io/QGIS-ResourceSharing/authoring/what-to-share.html) like processing scripts, styles and SVG images directly to QGIS from online repositories. Both of these plugins sport a custom graphical user interface:

![Examples of graphical plugins](images/graphical_plugin_examples.png)

Sometimes the plugins add a neat new functionality, like [adding a globe visualization](https://github.com/GispoCoding/GlobeBuilder) to the current project. If such functions are deemed useful enough, they might be intergrated natively to the program over time.

A custom user interface isn't a necessity for plugins. [Openlayers plugin](https://github.com/sourcepole/qgis-openlayers-plugin) has its main functionality (adding background maps) in a drop-down menu. [QNEAT3](https://root676.github.io/), a network analysis library, is a processing plugin: it adds a new provider and a bunch of scripts. The description of QNEAT3 mentions that some scripts require matplotlib Python plotting library: since QGIS includes a Python installation (on Windows), external libraries can easily be pip installed.

As the examples above show, QGIS plugins can extend the base program in many forms. However, these materials focus on ones with custom GUI's.

When is it beneficial to engage in plugin development? Plugin development can be time-consuming and not worth the while if you simply want to share a quick and dirty script with a co-worker. Models and processing scripts, too, are simpler than full GUI plugins since they have functionality to easily e.g. ask user for input and output paths. From the examples above, creating a GUI plugin might pay off when you have a service you want to reach more people or believe you can add a new necessary feature not possible with the existing tools.

## 2.2. QT framework: signals and slots
QGIS is built on a development framework called [QT](https://www.qt.io/). Basically all the user interface elements (buttons, tables, windows etc.) are based on QT objects.

A central QT concept is signals and slots. These two are used to connect what the user does to actions in the program: signal sends the information that something has been done and slot is the reaction. Signals can carry information related to the action made by the user or the state of the system, and then pass these to the slots, which are often Python functions. For a thorough discussion of these concepts, see [Nils Nolde's tutorial](https://gis-ops.com/qgis-3-plugin-tutorial-pyqt-signal-slot-explained/).

As an example of the signal/slot system in action, pressing _Zoom to layer_ ![Zoom to layer icon](images/zoom_to_layer.JPG)
sends a signal that the button has been clicked to the program and QGIS reacts presumably by calling ***iface.zoomToActiveLayer()*** behind the scenes.

Developers of plugins with GUI's must similarly think in terms of what each element in their interface does and how the user can interact with it. 

## 2.3. Elements of a QGIS plugin
QGIS plugins written in Python (which pretty much all _external_ plugins are) are stored in a directory, which should have a few required and a few recommended files. This list is from [The PyQGIS developer cookbook](https://docs.qgis.org/latest/en/docs/pyqgis_developer_cookbook/plugins/plugins.html#plugin-files):

In [ ]:
PYTHON_PLUGINS_PATH/
  MyPlugin/
    __init__.py    --> *Initialization code: required*
    metadata.txt   --> *Info about the plugin, its author etc.: required*
    mainPlugin.py  --> *Actual plugin code: can have multiple .py files*
    resources.qrc  --> *Resources, like paths to image files: likely useful*
    resources.py   --> *Python compiled resources, likely useful*
    form.ui        --> *Graphical user interface in XML format: likely useful*
    form.py        --> *Python compiled version of GUI, likely useful*

_Resources.qrc_ and _form.ui_ are both related to QT: the first is a list of paths to inform QT where things like icon image files are found and the second instructions on how to format the UI. Both of these must be compiled, encodes the e.g. image paths to images as binary data in .py files. See [Ujaval Gandi's tutorial section 9 for more info on compiling the files](http://www.qgistutorials.com/it/docs/3/building_a_python_plugin.html#procedure).

What about the plugin path? That's the path where QGIS loads the plugins from when the program is opened. The folder is found under the currently active profile's path. 

A shortcut to this path is available from QGIS interface: *Setting drop-down > User profiles > Open active profile folder*. From this folder navigate to *Python > Plugins*. Here you can find all your installed plugins. Check out a few of them! You may even open their main plugin .py files to see how other programmers have created their plugins.

The plugin path may for example look like this:
![Example plugins folder](images/plugins_folder.png)

## Challenge Y: Modifying a plugin
In the course materials folder under _data_, there's another [(zipped) folder called *cool_plugin*](https://github.com/csc-training/pyqgis/blob/main/data/cool_plugin.zip). Download the zipped folder and unzip it in your project folder.  Within the folder are files for a functional, albeit simple plugin. If you're interested in creating such a plugin from the scratch, [here's an earlier tutorial](https://autogis-site.readthedocs.io/en/latest/lessons/PyQGIS/pyqgis.html#developing-the-plugin) by the author.

#### Challenge preparations
First copy the whole folder and paste it to the active profile's _plugins_ folder (see above on how to locate this folder). 

Then save your current project and restart QGIS. This should load the Cool Plugin and add it under _Plugins > Cool plugin_ drop down menu and as a default icon in the toolbar ![Default icon](images/default_plugin_icon.JPG)

Running the plugin throws this GUI:

![Cool plugin interface](images/cool_plugin_interface.JPG)

Try pressing the magic button!
Yeah, doesn't do much. Your task is to add more functionality to the plugin. Start modifying the code in *plugins > Cool_plugin > cool_plugin.py*. You can do the editing in any text editor: like simply Notepad on Windows.

The function's we're interested in are at the bottom of the script. There's _coolFunction_ and _run_. 

In _run_ Magic Button is connected to the cool function – or, for the clicked signal, the _coolFunction_ slot is defined:

In [ ]:
self.dlg.magicButton.clicked.connect(self.coolFunction)

Translated to human speak, this means:

In [ ]:
Every time this button is pressed, run coolFunction

coolFunction simply throws a QT [messagebox](https://doc.qt.io/qt-5/qmessagebox.html) (the third parameter is the actual message).

In [ ]:
def coolFunction(self):
    """Does something cool (but currently just throws an messagebox)"""
    QMessageBox.information(self.dlg, "Cool message", "You just clicked a button")

FINALLY: install a plugin called **Plugin reloader** from _Plugins > Manage and install plugins_. This allows reloading changes made into the code without restarting QGIS every time. Once plugin reloader is installed, configure its settings so that it reinstalls _Cool plugin_.
![Cool plugin configuration](images/plugin_reloader_settings.JPG)

### CHALLENGE Y TASKS
1. Instead of giving out a static message, load in the current active layer and write the layer's name in the messagebox. If there're no layers in the project, write "No layers available!". 
    - Hint: Reference to the interface is saved as `self.iface`. 
    - What type of object does `activeLayer()` return when there are no layers?

2. Add a new GUI object by modifying *cool_plugin_dialog_base.ui* in _QT Designer_. QT designer is a graphical UI designer program, which should be installed alongside QGIS. [See these instuctions on adding GUI elements](https://autogis-site.readthedocs.io/en/latest/lessons/PyQGIS/pyqgis.html#adding-ui-elements).
    - Add another `Push button` with the text "Simplify".
    - Add a new function that processes geometry simplification on the active layer (if one is available).
    - Connect the button and the function.
    - Remember to import `Processing`

3. Add another GUI element. For example, add a new _QSpinBox_, which allows the user to input numerical values. Use the Spinbox input as the tolerance level for the simplification level.
    - GUI elements can be accessed through `self.dlg.elementName`. 
    - How to access the current value of the spinbox? [Hint](https://doc.qt.io/qt-5/qspinbox.html#value-prop)

The final plugin can, for example, look like this. You can [download this plugin from here](https://github.com/csc-training/pyqgis/blob/main/data/cool_plugin_SOLUTIONS.zip):

![Cool plugin final view](images/cool_plugin_final.JPG)

## 3. PyQGIS resources
Below are some great resources for delving into PyQGIS:

### General resources
[PyQGIS Developer Cookbook](https://docs.qgis.org/latest/en/docs/pyqgis_developer_cookbook/index.html), by [QGIS contributors](https://docs.qgis.org/3.16/en/docs/user_manual/preamble/contributors.html)

[PyQGIS Documentation](https://qgis.org/pyqgis/latest/index.html), by [QGIS contributors](https://docs.qgis.org/3.16/en/docs/user_manual/preamble/contributors.html)

[Customizing QGIS with Python (course)](https://courses.spatialthoughts.com/pyqgis-in-a-day.html), by Ujaval Gandhi, shared under CC BY-NC 4.0

[PyQGIS 101: Introduction to QGIS Python programming for non-programmers](https://anitagraser.com/pyqgis-101-introduction-to-qgis-python-programming-for-non-programmers/), by Anita Graser

[The PyQGIS Programmer's Guide 3](http://locatepress.com/ppg3) (available for purchase) by Gary Sherman

### Plugin development
For starting out with plugin development, see these resources:

[Building a Python Plugin (QGIS3)](http://www.qgistutorials.com/it/docs/3/building_a_python_plugin.html), by Ujaval Gandhi

[QGIS 3 Plugin Tutorial – Plugin development reference guide](https://gis-ops.com/qgis-3-plugin-tutorial-plugin-development-reference-guide/), by Nils Nolde